
# Clasificación con scikit-learn

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />


## 1. Clasificación

En clasificación, la etiqueta es discreta, por ejemplo Spam y No Spam. En otras palabras, se proporciona una distinción clara entre las categorías. Es más, es importante indicar que estas categorías son nominales y no ordinales. Las variables nominales y ordinales son ambas subcategorías de las variables categóricas. Las variables ordinales tienen asociado un orden, por ejemplo, las tallas de las camisetas "XL > L > M > S". Por el contrario, las variables nominales no implican un orden, por ejemplo, no podemos asumir (en general) "naranja > azul > verde".

Usaremos el dataset Iris, el cual fue recolectado por el estadístico y biólogo Ronal Fisher. El conjunto de datos contiene 50 muestras de cada una de tres especies de Iris (Iris setosa, Iris virginica e Iris versicolor). Se midió cuatro rasgos de cada muestra: lo largo y lo ancho del sépalos y pétalos, en centímetros. 

Cargaremos el dataset de Iris desde la librería scikit-learn. Al cargar el dataset notaremos que se incluye la meta-data y que las clases ya vienen convertidas en números

- 0=Iris-Setosa
- 1=Iris-Versicolor
- 2=Iris-Virginica.

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets

iris = datasets.load_iris()
iris.keys()

In [ ]:
iris_labels=iris.target_names
iris_labels

In [ ]:
iris

In [ ]:
# Creamos un dataframe a partir del dataset cargado desde scikit-learn

iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df["class"]=iris.target

print("Tamaño del dataset: ", len(iris_df))
iris_df.head()

In [ ]:
# Imprimimos los valores únicos que aparecen en el campo class

iris_df['class'].unique()

En aprendizaje supervisado se necesitan datos para entrenar el modelo y validarlo. Se suele hablar de tres categorías,

- training set (60% del dataset): Es usado para entrenar o construir el modelo. En esta fase se crean modelos usando multiples algoritmos para luego comparar sus respectivos desempeños.

- cross validation set (20% del dataset): Es usado para comparar el rendimiento de los diferentes modelos entrenados en la etapa anterior. Se emplea para elegir el modelo final.

- test set (20% del dataset): Es usado para determinar el rendimiento del modelo final. Se estima el error de la predicción del modelo.


Por simplicidad, en general se suelen dividir el dataset original en un training set y un test set. La regla esa usar un 70% para el training  y un 30% para el test.

Usaremos la función `train_test_split` de scikit-learn para generar el training y test.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(iris_df, test_size=0.3)

print("Tamaño del train set: ", len(df_train))
print("Tamaño del test set: ", len(df_test))

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
print("Numero de objetos por clase en dataset:\n", iris_df['class'].value_counts(sort=False))
print("Numero de objetos por clase en train set:\n", df_train['class'].value_counts(sort=False))
print("Numero de objetos por clase en test set:\n", df_test['class'].value_counts(sort=False))

In [ ]:
# Agregamos el parámetro stratify para conservar las proporciones del dataset original

df_train, df_test = train_test_split(iris_df, test_size=0.3, stratify=iris_df['class'])

In [ ]:
print("Numero de objetos por clase en dataset:\n", iris_df['class'].value_counts(sort=False))
print("Numero de objetos por clase en train set:\n", df_train['class'].value_counts(sort=False))
print("Numero de objetos por clase en test set:\n", df_test['class'].value_counts(sort=False))

Es conveniente ser explícito en que columnas serán las variables predictoras y la columna que corresponde a la predicción. Para ellos definiremos las variables X_train, y_train, X_test, y_test

Por simplicida, usaremos solamente las columnas petal_length y petal_width como features para construir el modelo. Estas columnas corresponden a los indices 2 y 3.

In [ ]:
X_train=np.array(df_train.iloc[:,2:4])
y_train=np.array(df_train.iloc[:,4])

X_test=np.array(df_test.iloc[:,2:4])
y_test=np.array(df_test.iloc[:,4])

In [ ]:
print("Min: ", X_train.min(axis=0))
print("Max: ", X_train.max(axis=0))
print("Mean: ", X_train.mean(axis=0))
print("Std Dev: ", X_train.std(axis=0))

Es necesario normalizar los features almacenados en X_train y Y_train. Para ellos usaramos la función `StandardScaler`

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
print("Min: ", X_train_std.min(axis=0))
print("Max: ", X_train_std.max(axis=0))
print("Mean: ", X_train_std.mean(axis=0))
print("Std Dev: ", X_train_std.std(axis=0))

In [ ]:
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

Definiremos una función llamada plot_decision_regions() la cual será usada graficar los resultados de la clasificación de los modelos

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
%matplotlib inline

# Función para graficar los bordes de decisión para datasets 2-dimensionales

def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')

    # highlight test samples
    if test_idx:
        # plot all samples
        X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    c='',
                    edgecolor='black',
                    alpha=1.0,
                    linewidth=1,
                    marker='o',
                    s=100, 
                    label='test set')

In [ ]:
X_combined_std.shape

In [ ]:
X_train_std.shape

## 1.1 Crear modelo de clasificación usando un Support Vector Classifier (SVC)

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=1.0, random_state=1)
svm.fit(X_train_std, y_train)

svm

In [ ]:
y_pred=svm.predict(X_test_std)
y_pred

In [ ]:
pd.DataFrame({"observado":y_test, "prediccion":y_pred})

In [ ]:
plot_decision_regions(X_combined_std, 
                      y_combined,
                      classifier=svm, 
                      test_idx=range(105, 150))

plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

Los símbolos de colores corresponden a las tres diferentes clases que existen en el dataset. Los círculos de color negro que encierran a los símbolos corresponde al test set, lo cual permite visualizar cuan bien funciona la separación.

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))

In [ ]:
# Función para imprimir la matriz de confusión usando etiquetas

def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [ ]:
print_cm(confusion_matrix(y_test, y_pred), iris_labels )

## 1.2 Crear modelo de clasificación usando un Regresión logística (LogisticRegression)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1)
lr.fit(X_train_std, y_train)

lr

In [ ]:
y_pred=lr.predict(X_test_std)
y_pred

In [ ]:
pd.DataFrame({"observado":y_test, "prediccion":y_pred})

In [ ]:
plot_decision_regions(X_combined_std, 
                      y_combined,
                      classifier=lr, 
                      test_idx=range(105, 150))

plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

In [ ]:
print_cm(confusion_matrix(y_test, y_pred), iris_labels )